In [86]:
import streamlit as st
import sqlite3 as sq
import requests as rq
import pandas as pd

import folium
import json
import os

Запросы для маршрутов

In [87]:
routes_from_to_url = 'http://irkbus.ru/php/getRoutes.php?city=irkutsk&info=12345&lang=ru&_=1686163028764'
stations_url = 'http://irkbus.ru/php/getStations.php?city=irkutsk&info=12345&lang=ru&_=1686163028767'

In [141]:
response = rq.get(stations_url)
stations_data = pd.DataFrame.from_dict(json.loads(response.text) if response.status_code == 200 else False).sort_values(by = 'id')

In [142]:
response = rq.get(routes_from_to_url)
routes_from_to_data = pd.DataFrame.from_dict(json.loads(response.text) if response.status_code == 200 else False).replace('Тр', 'Трамвай').replace('А', 'Автобус').replace('Т', 'Троллейбус').sort_values(by = 'id').reset_index().drop('index', axis = 1)

In [143]:
class Routes():
    def __init__(self,  routes_df : pd.DataFrame, stations_df : pd.DataFrame):
        self.routes_df =  routes_df
        self.stations_df =  stations_df


        self.routes_names_constraints = {'Avtobus_' : 'Автобус', ' Троллейбус' : 'Троллейбус'}

        self.routes_df.name = self.routes_df.name.apply(self.routes_names_correcter)
        stations_df.lat = stations_df.lat.apply(self.stations_cords_correcter)
        stations_df.lng = stations_df.lng.apply(self.stations_cords_correcter)

    def routes_names_correcter(self, name : str) -> str:
        for constraint in self.routes_names_constraints:
            if constraint in name:
                if constraint == None:
                    return('НЕ НАЙДЕНО')
                
                name = list(name)
                for let in constraint:
                    name.remove(let)
                
                name = ''.join(name) + f'_{self.routes_names_constraints[constraint]}'

        return(name)
    
    def stations_cords_correcter(self, cords : int) -> float:
        cords = str(cords)
        cords = list(cords)

        if '.' not in cords:
            if cords[0] == '1':
                cords.insert(3, '.')
                cords = float(''.join(cords))
            
            else:
                cords.insert(2, '.')
                cords = float(''.join(cords))
        
        return(cords)



    def route_data(self, route_name : str) -> pd.DataFrame:
        try:
            return(self.routes_df[self.routes_df['name'] == route_name])
        
        except:
            return(False)
        
    
    def get_stations(self, route_name : str) -> list:
        # print(self.routes_df[self.routes_df['name'] == route_name][['id', 'name', 'type', 'num', 'fromst', 'fromstid', 'tost', 'tostid']])
        try:
            return(self.routes_df[['id', 'name', 'type', 'num', 'fromst', 'fromstid', 'tost', 'tostid'], self.routes_df['name'] == route_name])
        
        except:
            return(False)

---

In [144]:
routes_manipulator = Routes(routes_from_to_data, stations_data)

52.28302
52.273956
52.285909
52.283472
52.283113
52.286533
52.283451
52.286406
52.277013
52.281943
52.277072
52.277155
52.274627
52.273537
52.276759
52.274701
52.27529
52.277118
52.274647
52.277166
52.272248
52.272192
52.278853
52.279161
52.264233
52.280028
52.264269
52.267626
52.279901
52.267578
52.279921
52.27979
52.265078
52.26541
52.279714
52.279823
52.267342
52.267266
52.279982
52.279872
52.271847
52.271895
52.28074
52.280843
52.275133
52.259777
52.275694
52.27866
52.259368
52.278604
52.252828
52.282796
52.256454
52.256572
52.282695
52.261702
52.284618
52.261443
52.284997
52.28657
52.262768
52.286447
52.263073
52.265125
52.282214
52.265227
52.281953
52.2814
52.265015
52.27695
52.264867
52.276807
52.264736
52.287597
52.264633
52.287381
52.264242
52.282536
52.264198
52.267581
52.28191
52.279416
52.267129
52.27918
52.265127
52.276267
52.264687
52.261743
52.276377
52.273897
52.325299
52.247063
52.273661
52.27003
52.269825
52.323091
52.266263
52.322791
52.266007
52.317443
52.263341
52.

In [145]:
routes_manipulator.route_data('1Усолье')

,id,name,type,num,fromst,fromstid,tost,tostid
22,13,1Усолье,Трамвай,1,Трампарк,137,9 столовая,133
24,14,1Усолье,Трамвай,1,9 столовая,133,Трампарк,128


In [146]:
routes_manipulator.stations_df

,id,name,descr,lat,lng,type
826,1,Степана Разина,в сторону ж/д вокзала,52.283020,104.277680,1
44,1,Аэропорт,конечная 4 троллейбуса,52.273956,104.355826,0
503,2,Сквер им. Кирова,в сторону ул. Карла Маркса,52.285909,104.282473,0
825,2,Степана Разина,в сторону центрального рынка,52.283472,104.277002,1
748,3,Художественный музей,в сторону пл. Кирова,52.283113,104.280979,0
...,...,...,...,...,...,...
603,1028,СНТ Радуга-2,из города,52.102024,104.343842,0
604,1029,СНТ Радуга-2,в город,52.101687,104.344614,0
576,1030,СНТ Минерал,из города,52.093745,104.352436,0
577,1031,СНТ Минерал,в город,52.093350,104.353004,0
